In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
df1 = pd.read_csv('./Datasets_final/q2/train.csv',dtype=str)
df2 = pd.read_csv('./Datasets_final/q2/test.csv',dtype=str)

In [3]:
def append_ext(fn):
    return fn+".jpg"

In [4]:
df1['image_file'] = df1['image_file'].apply(append_ext)
df2['image_file'] = df2['image_file'].apply(append_ext)

# Create train_datagen and test_datagen

In [5]:
# this is the augmentation configuration we will use for training
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.25)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# Initialize paths where images flow from

In [1]:
train_generator = datagen.flow_from_dataframe(
        dataframe = df1,
        directory = './Datasets_final/q2/train',
        x_col = 'image_file',
        y_col = 'emotion',
        subset="training",
        batch_size = 16,
        class_mode = 'categorical',
        target_size = (150, 150))

val_generator = datagen.flow_from_dataframe(
        dataframe = df1,
        directory = './Datasets_final/q2/train',
        x_col = 'image_file',
        y_col = 'emotion',
        subset="validation",
        batch_size = 16,
        class_mode = 'categorical',
        target_size = (150, 150))

test_generator = test_datagen.flow_from_dataframe(
        dataframe = df2,
        directory = './Datasets_final/q2/test',
        x_col = 'image_file',
        y_col = None,
        batch_size = 16,
        class_mode = None,
        target_size = (150, 150))

NameError: name 'datagen' is not defined

# Inializing the CNN


In [4]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(activation = 'relu', units = 128))
classifier.add(Dense(activation = 'sigmoid', units = 5))
classifier.compile (optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 41472)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               5308544   
_________________________________________________________________
dense_2 (Dense)              (None, 5)                

In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

classifier.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=40
)

Epoch 1/40
91/91 [==============================] - 24s 266ms/step - loss: 1.7422 - accuracy: 0.3462 - val_loss: 1.7099 - val_accuracy: 0.0333
Epoch 2/40
91/91 [==============================] - 24s 267ms/step - loss: 1.3080 - accuracy: 0.4526 - val_loss: 2.4180 - val_accuracy: 0.2217
Epoch 3/40
91/91 [==============================] - 24s 265ms/step - loss: 1.0866 - accuracy: 0.5625 - val_loss: 2.2838 - val_accuracy: 0.3710
Epoch 4/40
91/91 [==============================] - 24s 267ms/step - loss: 0.9272 - accuracy: 0.6264 - val_loss: 1.8301 - val_accuracy: 0.2537
Epoch 5/40
91/91 [==============================] - 24s 268ms/step - loss: 0.7827 - accuracy: 0.6875 - val_loss: 1.7489 - val_accuracy: 0.2857
Epoch 6/40
91/91 [==============================] - 24s 268ms/step - loss: 0.6579 - accuracy: 0.7404 - val_loss: 2.2977 - val_accuracy: 0.2623
Epoch 7/40
91/91 [==============================] - 24s 265ms/step - loss: 0.6089 - accuracy: 0.7782 - val_loss: 2.9822 - val_accuracy: 0.1919

In [11]:
classifier.evaluate_generator(generator=val_generator, steps=STEP_SIZE_TEST)

[8.853296279907227, 0.255159467458725]

In [12]:
pred=classifier.predict_generator(test_generator, steps=STEP_SIZE_TEST+1, verbose=1)

35/35 [==============================] - 10s 296ms/step


In [13]:
predicted_class_indices=np.argmax(pred,axis=1)

In [15]:
pd.DataFrame(predicted_class_indices, columns=['emotion']).to_csv("./Datasets_final/q2/final_submission2.csv", index=False)
